In [12]:
!pip install pydub

/bin/bash: /home/staticowl/miniconda3//lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import librosa
import librosa.display
import soundfile as sf
import os

# Load and preprocess dataset

# You may need to use librosa or other audio processing libraries for feature extraction

# Model architecture (Variational Autoencoder for simplicity)
class VoiceConversionModel(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VoiceConversionModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Linear(512, input_dim),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Define hyperparameters
input_dim = 128 
latent_dim = 64  


model = VoiceConversionModel(input_dim, latent_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    inputs = torch.tensor(X_train, dtype=torch.float32)
    targets = torch.tensor(Y_train, dtype=torch.float32)
    optimizer.zero_grad()
    _, outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

input_feature = torch.tensor(X_train[0], dtype=torch.float32)
_, converted_feature_raw = model(input_feature)
converted_feature = converted_feature_raw.detach().numpy()
print("Converted Feature:", converted_feature)



Epoch [100/1000], Loss: 0.0675
Epoch [200/1000], Loss: 0.0145
Epoch [300/1000], Loss: 0.0004
Epoch [400/1000], Loss: 0.0000
Epoch [500/1000], Loss: 0.0000
Epoch [600/1000], Loss: 0.0001
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000
Converted Feature: [0.51578826 0.2001884  0.69031364 0.02166189 0.5159407  0.3202289
 0.8360705  0.53537214 0.7203905  0.9234433  0.65337014 0.84394044
 0.5880028  0.18579386 0.64804876 0.13330191 0.7680859  0.93606985
 0.8281777  0.10790095 0.34701195 0.7901224  0.21725005 0.27404773
 0.1890176  0.0885486  0.5766597  0.30678824 0.35581806 0.14792757
 0.71813244 0.4264056  0.4423367  0.3240838  0.70782644 0.3428624
 0.86315185 0.46705362 0.96288645 0.48173457 0.7110623  0.78102577
 0.60397464 0.16133423 0.22240531 0.5163418  0.2355866  0.34963
 0.9228073  0.36901793 0.15199684 0.3641538  0.5419421  0.12549558
 0.14360194 0.08012033 0.7282584  0.5747153  0.6133456  0.2168256
 0.478

In [20]:
from pydub import AudioSegment

def spectrogram_to_audio(spectrogram):
    # Inverse transform spectrogram to waveform using Griffin-Lim

    print("Shape of spectrogram:", spectrogram.shape)
    print("Data type of spectrogram:", spectrogram.dtype)
    waveform = librosa.griffinlim(np.abs(spectrogram))
    return waveform

converted_audio = spectrogram_to_audio(converted_feature)
sf.write("output.wav", converted_audio, 128)

Shape of spectrogram: (128,)
Data type of spectrogram: float32


IndexError: tuple index out of range